In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import math as m

In [2]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

In [3]:
df = pd.read_csv(r'E:\NEU\ML Ops\Project\StockPricePrediction\data\final_dataset.csv')

c:\Users\kandi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8,9,10,11,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# df = pd.read_csv('final_dataset.csv')

In [5]:
df

,date,Mkt-RF,SMB,HML,RMW,CMA,RF,DGS10,RIFSPPFAAD90NB,DGS1,...,DCOILBRENTEU,ads_index,open,high,low,close,volume,dividends,stock_splits,SP500
0,1854-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1854-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1854-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1854-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1854-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62051,2024-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,234.449997,236.850006,234.449997,236.479996,36254500.0,0.0,0.0,NaN
62052,2024-10-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,233.889999,236.220001,232.600006,235.860001,38846600.0,0.0,0.0,NaN
62053,2024-10-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,234.080002,235.139999,227.759995,230.759995,52287000.0,0.0,0.0,NaN
62054,2024-10-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,229.979996,230.820007,228.410004,230.570007,31109500.0,0.0,0.0,NaN


In [6]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.set_index('date', inplace=True)

In [7]:
# Step 2: Slice the data to include only dates after 2004-01-01
df = df.loc['2004-01-01':]

In [8]:
# List of columns to check for NaN values
columns_to_check = ['open', 'high', 'low', 'close', 'volume', 'SP500']

# Remove rows where all specified columns are NaN
df_cleaned = df.dropna(subset=columns_to_check, how='all')

In [9]:
df.describe()

,Mkt-RF,SMB,HML,RMW,CMA,RF,USRECDP,USRECD,USRECDM,ads_index,open,high,low,close,volume,dividends,stock_splits,SP500
count,5201.000000,5201.000000,5201.000000,5201.000000,5201.000000,5201.000000,7599.000000,7599.000000,7599.000000,7591.000000,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03,1258.000000,1258.000000,2512.000000
mean,0.040711,0.000535,-0.001181,0.016435,-0.000094,0.005839,0.080011,0.080011,0.080274,-0.250928,146.176032,147.800565,144.654762,146.308836,9.266660e+07,0.003549,0.003180,3280.603420
std,1.208168,0.631346,0.787480,0.453615,0.380189,0.007202,0.271327,0.271327,0.271734,2.071278,41.786013,42.021225,41.525784,41.786841,5.291959e+07,0.028021,0.112777,1034.017927
min,-12.000000,-4.550000,-4.970000,-2.720000,-2.740000,0.000000,0.000000,0.000000,0.000000,-26.524400,55.472059,55.574208,51.709551,54.569736,2.404830e+07,0.000000,0.000000,1829.080000
25%,-0.430000,-0.360000,-0.330000,-0.230000,-0.200000,0.000000,0.000000,0.000000,0.000000,-0.372406,122.872769,124.133498,121.650088,123.003281,5.761422e+07,0.000000,0.000000,2381.642500
50%,0.070000,0.000000,-0.020000,0.010000,-0.010000,0.001000,0.000000,0.000000,0.000000,-0.095002,147.627858,149.251860,146.196051,147.771301,7.875405e+07,0.000000,0.000000,2951.235000
75%,0.590000,0.350000,0.310000,0.250000,0.190000,0.009000,0.000000,0.000000,0.000000,0.151029,173.652050,175.598158,172.271674,173.622738,1.093357e+08,0.000000,0.000000,4155.500000
max,11.350000,5.720000,6.730000,4.210000,2.520000,0.022000,1.000000,1.000000,1.000000,9.440280,236.206595,237.490005,234.449997,236.479996,4.265100e+08,0.250000,4.000000,5864.670000


In [10]:
df_cleaned.shape

(2517, 42)

In [11]:
# Drop columns that contain only NaN and 0.0 values
df = df.drop(columns=[col for col in df.columns if df[col].isna().all() or (df[col] == 0.0).all()])

In [12]:
df = df.apply(lambda col: pd.to_numeric(col.replace('.', np.nan), errors='coerce')).fillna(0.0)

In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7604 entries, 2004-01-01 to 2024-10-25
Data columns (total 42 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Mkt-RF          7604 non-null   float64
 1   SMB             7604 non-null   float64
 2   HML             7604 non-null   float64
 3   RMW             7604 non-null   float64
 4   CMA             7604 non-null   float64
 5   RF              7604 non-null   float64
 6   DGS10           7604 non-null   float64
 7   RIFSPPFAAD90NB  7604 non-null   float64
 8   DGS1            7604 non-null   float64
 9   CBETHUSD        7604 non-null   float64
 10  DBAA            7604 non-null   float64
 11  USRECDP         7604 non-null   float64
 12  USRECD          7604 non-null   float64
 13  DCPN3M          7604 non-null   float64
 14  DCPF1M          7604 non-null   float64
 15  DAAA            7604 non-null   float64
 16  T10Y3M          7604 non-null   float64
 17  BAMLH0A0HYM2   

In [14]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to a new line

In [15]:
# Drop the 'dividends' and 'stock_splits' columns
df = df.drop(columns=['dividends', 'stock_splits'])

In [16]:
# List of columns to check for NaN values
columns_to_check = ['open', 'high', 'low', 'close', 'volume', 'SP500']

# Remove rows where all specified columns are NaN
df_cleaned = df.dropna(subset=columns_to_check, how='all')

In [17]:
df_cleaned.shape

(7604, 40)

In [18]:
# Step 3: Remove columns with only 0.0 and NaN values
df_cleaned= df_cleaned.loc[:, ~(df_cleaned.isna().all() | (df_cleaned == 0.0).all())]

# Display the cleaned DataFrame
# print(df_cleaned)

In [19]:
# Check the data types of all columns
print(df_cleaned.dtypes)

Mkt-RF            float64
SMB               float64
HML               float64
RMW               float64
CMA               float64
RF                float64
DGS10             float64
RIFSPPFAAD90NB    float64
DGS1              float64
CBETHUSD          float64
DBAA              float64
USRECDP           float64
USRECD            float64
DCPN3M            float64
DCPF1M            float64
DAAA              float64
T10Y3M            float64
BAMLH0A0HYM2      float64
T10YIE            float64
DHHNGSP           float64
AMERIBOR          float64
USRECDM           float64
CBBTCUSD          float64
DEXUSUK           float64
DCOILWTICO        float64
VIXCLS            float64
DEXJPUS           float64
DEXUSEU           float64
T5YIE             float64
OBMMIJUMBO30YF    float64
NIKKEI225         float64
BAMLH0A0HYM2EY    float64
DCOILBRENTEU      float64
ads_index         float64
open              float64
high              float64
low               float64
close             float64
volume      

In [20]:
df_cleaned["tomorrow"] = df_cleaned["close"].shift(-1)

In [21]:
df_cleaned.shape

(7604, 41)

In [22]:
# List of columns to check for NaN values
columns_to_check = ['open', 'high', 'low', 'close', 'volume', 'SP500']
# Step 2: Remove rows where any of the specified columns are 0.0
df_cleaned = df_cleaned[~df_cleaned[columns_to_check].isin([0.0]).any(axis=1)]

# Display the cleaned DataFrame
print(df_cleaned)

            Mkt-RF   SMB   HML   RMW   CMA     RF  DGS10  RIFSPPFAAD90NB  DGS1  CBETHUSD  DBAA  USRECDP  USRECD  DCPN3M  DCPF1M  DAAA  T10Y3M  BAMLH0A0HYM2  T10YIE  DHHNGSP  AMERIBOR  USRECDM  CBBTCUSD  DEXUSUK  DCOILWTICO  VIXCLS  DEXJPUS  DEXUSEU  T5YIE  OBMMIJUMBO30YF  NIKKEI225  BAMLH0A0HYM2EY  DCOILBRENTEU  ads_index        open        high         low       close       volume    SP500    tomorrow
date                                                                                                                                                                                                                                                                                                                                                                                                                 
2019-10-28    0.62  0.28 -0.24  0.01 -0.04  0.007   1.85            1.77  1.60    181.87  4.01      0.0     0.0    1.78    0.00  3.08    0.20          3.85    1.64     2.49  1.860921      

In [23]:
df_cleaned.shape

(1253, 41)

In [24]:
df_cleaned['close_lag1'] = df_cleaned['close'].shift(1)
df_cleaned['close_lag2'] = df_cleaned['close'].shift(2)

In [25]:
y = df_cleaned['tomorrow']  # This is what you want to predict

# Define the features (X), including lagged close prices and other columns
X = df_cleaned[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'DGS10', 'RIFSPPFAAD90NB',
         'DGS1', 'CBETHUSD', 'DBAA', 'USRECDP', 'USRECD', 'DCPN3M', 'DCPF1M',
         'DAAA', 'T10Y3M', 'BAMLH0A0HYM2', 'T10YIE', 'DHHNGSP', 'AMERIBOR',
         'USRECDM', 'CBBTCUSD', 'DEXUSUK', 'DCOILWTICO', 'VIXCLS', 'DEXJPUS',
         'DEXUSEU', 'T5YIE', 'OBMMIJUMBO30YF', 'NIKKEI225', 'BAMLH0A0HYM2EY',
         'DCOILBRENTEU', 'ads_index', 'open', 'high', 'low', 'volume', 'SP500',
         'close', 'close_lag1', 'close_lag2']]  

In [26]:
column_list = X.columns.tolist()
print(column_list)

['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'DGS10', 'RIFSPPFAAD90NB', 'DGS1', 'CBETHUSD', 'DBAA', 'USRECDP', 'USRECD', 'DCPN3M', 'DCPF1M', 'DAAA', 'T10Y3M', 'BAMLH0A0HYM2', 'T10YIE', 'DHHNGSP', 'AMERIBOR', 'USRECDM', 'CBBTCUSD', 'DEXUSUK', 'DCOILWTICO', 'VIXCLS', 'DEXJPUS', 'DEXUSEU', 'T5YIE', 'OBMMIJUMBO30YF', 'NIKKEI225', 'BAMLH0A0HYM2EY', 'DCOILBRENTEU', 'ads_index', 'open', 'high', 'low', 'volume', 'SP500', 'close', 'close_lag1', 'close_lag2']


In [27]:
X[['close_lag1', 'close_lag2']] = X[['close_lag1', 'close_lag2']].fillna(0)

c:\Users\kandi\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [28]:
X1 = df_cleaned[['Mkt-RF', 'SP500', 'VIXCLS', 'DEXJPUS', 'NIKKEI225', 'T10Y3M', 'ads_index',
                'open', 'high', 'low', 'close']]

In [29]:
X_cleaned = X1.copy()

# Convert any remaining `datetime64` columns to Unix timestamp format as integers
for col in X_cleaned.columns:
    if X_cleaned[col].dtype == 'datetime64[ns]':
        X_cleaned[col] = X_cleaned[col].astype('int64') // 10**9  # Convert to Unix timestamp (seconds)

# Check all data types to ensure compatibility
print("Data types after explicit conversion:")
print(X_cleaned.dtypes)


Data types after explicit conversion:
Mkt-RF       float64
SP500        float64
VIXCLS       float64
DEXJPUS      float64
NIKKEI225    float64
T10Y3M       float64
ads_index    float64
open         float64
high         float64
low          float64
close        float64
dtype: object


In [30]:
# Save the DataFrame to a CSV file
X_cleaned.to_csv("cleaned_data.csv", index=False)

# Reload the data from CSV
import pandas as pd
X_cleaned_reloaded = pd.read_csv("cleaned_data.csv")

# Confirm that no timestamp types remain after reloading
print("Data types after reloading from CSV:")
print(X_cleaned_reloaded.dtypes)


Data types after reloading from CSV:
Mkt-RF       float64
SP500        float64
VIXCLS       float64
DEXJPUS      float64
NIKKEI225    float64
T10Y3M       float64
ads_index    float64
open         float64
high         float64
low          float64
close        float64
dtype: object


In [31]:
import tensorflow_data_validation as tfdv

# Generate statistics from the reloaded DataFrame
data_stats = tfdv.generate_statistics_from_dataframe(X_cleaned_reloaded)

# Infer schema from the generated statistics
schema = tfdv.infer_schema(statistics=data_stats)

tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Mkt-RF',FLOAT,required,,-
'SP500',FLOAT,required,,-
'VIXCLS',FLOAT,required,,-
'DEXJPUS',FLOAT,required,,-
'NIKKEI225',FLOAT,required,,-
'T10Y3M',FLOAT,required,,-
'ads_index',FLOAT,required,,-
'open',FLOAT,required,,-
'high',FLOAT,required,,-


In [32]:
# Validate a new batch of data against the schema
validation_result = tfdv.validate_statistics(data_stats, schema)
tfdv.display_anomalies(validation_result)


**Data Schema Generation:** You generated the schema using TFDV, which describes the types, presence, and expected properties of each feature.

**Data Statistics Generation:** You generated statistics from the data, giving you insights into the distribution, mean, standard deviation, and other statistics for each feature.

**Data Validation:**  You validated the current data against the generated schema. The “No anomalies found” message indicates that the data conforms to the schema, with no issues in types, missing values, or ranges.